In [1]:
import sys

# Absolute path to the folder you want to add
path_to_add = '/Users/jonnypreuss/Documents/03_Code/03_Pers/03_French_Verbs/french_verb_learning'

# Add it to the Python path if not already present
if path_to_add not in sys.path:
    sys.path.append(path_to_add)

# Now you can import modules from that folder or load data using relative paths from it


In [2]:
from src import config as con
from src import load_data as load
from src import select_input as input

2025-06-01 14:39:36.120 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager


In [3]:
import pandas as pd
from openpyxl import load_workbook
import random

In [15]:
EXCEL_FILE = "../data/Top_1000_verbs_French_USE.xlsx"

In [16]:
def load_workbook_sheet():
    wb = load_workbook(EXCEL_FILE)
    ws = wb.active
    return wb, ws

In [17]:
wb, ws = load_workbook_sheet()

In [18]:
wb

In [19]:
ws

<Worksheet "Solutions">

In [20]:
def get_random_task(ws):
    rows = range(con.START_ROW, ws.max_row + 1)
    # get those rows that have not yet been completed
    available_rows = [r for r in rows if str(ws[f"{con.STATUS_COL}{r}"].value).strip().lower() != "true"]
    if not available_rows:
        return None, None, None, None

    row = random.choice(available_rows)
    col = random.choice(con.CONJUGATION_COLS)

    # select the verb from the randomly selected row
    verb = ws[f"{con.VERB_COL}{row}"].value
    tense = ws[f"{col}1"].value
    subject = ws[f"{col}2"].value
    return row, col, verb, f"{tense} — {subject}"

In [21]:
print(ws.max_row)
rows = range(con.START_ROW, ws.max_row + 1)

1002


In [22]:
available_rows = [r for r in rows if str(ws[f"{con.STATUS_COL}{r}"].value).strip().lower() != "true"]

In [23]:
row, col, verb, prompt = get_random_task(ws)

In [24]:
row

573

In [25]:
col

'AR'

In [26]:
verb

'repousser'

In [27]:
prompt

'Conditionnel — nous'

In [28]:
import mlconjug3
from collections.abc import Mapping
from collections import defaultdict

In [30]:
# Load your Excel verb list
df = pd.read_excel("../data_prep/Top_1000_verbs_French_empty.xlsx", sheet_name="Top1000")
verbs = df["French"].dropna().unique()
verbs[:2]

array(['être', 'avoir'], dtype=object)

In [31]:
# Conjugator
conjugator = mlconjug3.Conjugator(language='fr')

# Standard subject order
pronoun_order = ['je', 'tu', 'il (elle, on)', 'nous', 'vous', 'ils (elles)']
imperative_order = ['tu', 'nous', 'vous']

# Subtenses to skip
skip_subtenses = {
    "Indicatif": ["Passé Simple"],
    "Subjonctif": ["Imparfait"]
}

# Output containers
rows = []
column_tracker = defaultdict(set)
form_columns = [
    "Form__participle_present",
    "Form__participle_past",
    "Form__participle_past_agreement",
    "Form__imperative"
]

In [32]:
i=0
for verb in verbs[:2]:
    try:
        print("--------------BLOCK 1---------------")
        conj = conjugator.conjugate(verb)
        print(conj.conjug_info.items())
        flat = {"verb": verb}
        print(flat)
        print()

        # Extract Participe forms
        print("--------------BLOCK 2---------------")
        participe_data = conj.conjug_info.get("Participe", {})
        print(participe_data)
        flat["Form__participle_present"] = participe_data.get("Participe Présent", "-")
        print(flat["Form__participle_present"])
        print()

        print("--------------BLOCK 3---------------")
        pp_dict = participe_data.get("Participe Passé", {})
        print(pp_dict)
        flat["Form__participle_past"] = pp_dict.get("masculin singulier", "-")
        print(flat["Form__participle_past"])
        print()

        print("--------------BLOCK 4---------------")
        has_agreement = any(pp_dict.get(k) not in [None, "-", ""] for k in["masculin pluriel", "feminin singulier", "feminin pluriel"])
        print(has_agreement)
        flat["Form__participle_past_agreement"] = "+ /-e/-s/-es" if has_agreement else "-"
        print(flat["Form__participle_past_agreement"])
        print()

        print("--------------BLOCK 5---------------")
        # --- Imperatif Handling (simple: always one form, use 'vous') ---
        imperatif_data = conj.conjug_info.get("Imperatif", {}).get("Imperatif Présent", {})
        print(imperatif_data)
        imperative_form = imperatif_data.get("", None)
        print(imperative_form)
        flat["Form__imperative"] = imperative_form if imperative_form else "-"
        print(flat["Form__imperative"])
        print()

        print("--------------MOODS BLOCK---------------")
        # Handle moods
        for mood, mood_data in conj.conjug_info.items():
            print(f"Mood: {mood}")
            print(f"Mood Data: {mood_data}")

            # Case 1: mood → subtense → pronoun
            # checks if mood_data is a dictionary and if every value in it is also a dict
            # because in that case the mood value would have subtenses
            if isinstance(mood_data, Mapping) and all(isinstance(v, Mapping) for v in mood_data.values()):
                print("Case 1")
                # now we check those subtenses and the corresponding conjugations
                for subtense, persons in mood_data.items():
                    print(f"Subtense: {subtense}")
                    # print(persons)
                    # persons is just the dict with "je": "conjugtion1"...and so on
                    # print(subtense.get(mood))
                    if subtense in skip_subtenses.get(mood, []):
                        continue
                    for pronoun in pronoun_order:
                        if pronoun in persons:
                            col = f"{mood}__{subtense}__{pronoun}"
                            # print(f"Col: {col}")
                            flat[col] = persons[pronoun]
                            # print(f"flat[col]: {flat[col]}")
                            column_tracker[(mood, subtense)].add(pronoun)

            # Case 2: Imperatif or other flat tense
            elif isinstance(mood_data, Mapping):
                print("Case 2")
                for subtense, persons in mood_data.items():
                    for pronoun in imperative_order:
                        if pronoun in persons:
                            col = f"{mood}__{subtense}__{pronoun}"
                            flat[col] = persons[pronoun]
                            column_tracker[(mood, subtense)].add(pronoun)

        rows.append(flat)
        print(f"ROWS!!! {rows[i]}")
        i+=1

    except Exception as e:
        print(f"❌ Could not conjugate '{verb}': {e}")

--------------BLOCK 1---------------
odict_items([('Infinitif', OrderedDict({'Infinitif Présent': 'être'})), ('Indicatif', OrderedDict({'Présent': OrderedDict({'je': 'suis', 'tu': 'es', 'il (elle, on)': 'est', 'nous': 'sommes', 'vous': 'êtes', 'ils (elles)': 'sont'}), 'Imparfait': OrderedDict({'je': 'étais', 'tu': 'étais', 'il (elle, on)': 'était', 'nous': 'étions', 'vous': 'étiez', 'ils (elles)': 'étaient'}), 'Futur': OrderedDict({'je': 'serai', 'tu': 'seras', 'il (elle, on)': 'sera', 'nous': 'serons', 'vous': 'serez', 'ils (elles)': 'seront'}), 'Passé Simple': OrderedDict({'je': 'fus', 'tu': 'fus', 'il (elle, on)': 'fut', 'nous': 'fûmes', 'vous': 'fûtes', 'ils (elles)': 'furent'})})), ('Conditionnel', OrderedDict({'Présent': OrderedDict({'je': 'serais', 'tu': 'serais', 'il (elle, on)': 'serait', 'nous': 'serions', 'vous': 'seriez', 'ils (elles)': 'seraient'})})), ('Subjonctif', OrderedDict({'Présent': OrderedDict({'je': 'sois', 'tu': 'sois', 'il (elle, on)': 'soit', 'nous': 'soyons',

In [21]:
rows

[{'verb': 'être',
  'Form__participle_present': 'étant',
  'Form__participle_past': 'été',
  'Form__participle_past_agreement': '-',
  'Form__imperative': 'soyez',
  'Indicatif__Présent__je': 'suis',
  'Indicatif__Présent__tu': 'es',
  'Indicatif__Présent__il (elle, on)': 'est',
  'Indicatif__Présent__nous': 'sommes',
  'Indicatif__Présent__vous': 'êtes',
  'Indicatif__Présent__ils (elles)': 'sont',
  'Indicatif__Imparfait__je': 'étais',
  'Indicatif__Imparfait__tu': 'étais',
  'Indicatif__Imparfait__il (elle, on)': 'était',
  'Indicatif__Imparfait__nous': 'étions',
  'Indicatif__Imparfait__vous': 'étiez',
  'Indicatif__Imparfait__ils (elles)': 'étaient',
  'Indicatif__Futur__je': 'serai',
  'Indicatif__Futur__tu': 'seras',
  'Indicatif__Futur__il (elle, on)': 'sera',
  'Indicatif__Futur__nous': 'serons',
  'Indicatif__Futur__vous': 'serez',
  'Indicatif__Futur__ils (elles)': 'seront',
  'Conditionnel__Présent__je': 'serais',
  'Conditionnel__Présent__tu': 'serais',
  'Conditionnel__Pr

In [22]:
df_out = pd.DataFrame(rows)

In [23]:
df_out

,verb,Form__participle_present,Form__participle_past,Form__participle_past_agreement,Form__imperative,Indicatif__Présent__je,Indicatif__Présent__tu,"Indicatif__Présent__il (elle, on)",Indicatif__Présent__nous,Indicatif__Présent__vous,...,"Conditionnel__Présent__il (elle, on)",Conditionnel__Présent__nous,Conditionnel__Présent__vous,Conditionnel__Présent__ils (elles),Subjonctif__Présent__je,Subjonctif__Présent__tu,"Subjonctif__Présent__il (elle, on)",Subjonctif__Présent__nous,Subjonctif__Présent__vous,Subjonctif__Présent__ils (elles)
0,être,étant,été,-,soyez,suis,es,est,sommes,êtes,...,serait,serions,seriez,seraient,sois,sois,soit,soyons,soyez,soient
1,avoir,ayant,eu,+ /-e/-s/-es,ayez,ai,as,a,avons,avez,...,aurait,aurions,auriez,auraient,aie,aies,ait,ayons,ayez,aient


In [39]:
# Build ordered columns
ordered_columns = ["verb"]

for (mood, subtense), pronouns in sorted(column_tracker.items()):
    print((mood, subtense))
    print(pronouns)
    if "imperatif" in mood.lower():
        subject_list = imperative_order
    else:
        subject_list = pronoun_order

    for pronoun in subject_list:
        if pronoun in pronouns:
            col = f"{mood}__{subtense}__{pronoun}"
            ordered_columns.append(col)

ordered_columns.extend(form_columns)

('Conditionnel', 'Présent')
{'tu', 'ils (elles)', 'nous', 'vous', 'je', 'il (elle, on)'}
('Indicatif', 'Futur')
{'tu', 'ils (elles)', 'nous', 'vous', 'je', 'il (elle, on)'}
('Indicatif', 'Imparfait')
{'tu', 'ils (elles)', 'nous', 'vous', 'je', 'il (elle, on)'}
('Indicatif', 'Présent')
{'tu', 'ils (elles)', 'nous', 'vous', 'je', 'il (elle, on)'}
('Subjonctif', 'Présent')
{'tu', 'ils (elles)', 'nous', 'vous', 'je', 'il (elle, on)'}


In [37]:
ordered_columns

['verb',
 'Conditionnel__Présent__je',
 'Conditionnel__Présent__tu',
 'Conditionnel__Présent__il (elle, on)',
 'Conditionnel__Présent__nous',
 'Conditionnel__Présent__vous',
 'Conditionnel__Présent__ils (elles)',
 'Indicatif__Futur__je',
 'Indicatif__Futur__tu',
 'Indicatif__Futur__il (elle, on)',
 'Indicatif__Futur__nous',
 'Indicatif__Futur__vous',
 'Indicatif__Futur__ils (elles)',
 'Indicatif__Imparfait__je',
 'Indicatif__Imparfait__tu',
 'Indicatif__Imparfait__il (elle, on)',
 'Indicatif__Imparfait__nous',
 'Indicatif__Imparfait__vous',
 'Indicatif__Imparfait__ils (elles)',
 'Indicatif__Présent__je',
 'Indicatif__Présent__tu',
 'Indicatif__Présent__il (elle, on)',
 'Indicatif__Présent__nous',
 'Indicatif__Présent__vous',
 'Indicatif__Présent__ils (elles)',
 'Subjonctif__Présent__je',
 'Subjonctif__Présent__tu',
 'Subjonctif__Présent__il (elle, on)',
 'Subjonctif__Présent__nous',
 'Subjonctif__Présent__vous',
 'Subjonctif__Présent__ils (elles)',
 'Form__participle_present',
 'Form__

In [ ]:

# Final output
df_out = pd.DataFrame(rows)
df_out = df_out.reindex(columns=ordered_columns).fillna("-")